### I. Set up

In [1]:
%pip install pandas
%pip install plotly
%pip install nbformat
import sys
!{sys.executable} -m pip install nbformat --upgrade
%pip install ace_tools_open


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import libraries
# Required packages: plotly, pandas
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import defaultdict
import ace_tools_open as tools

import level_2_learning_comparisons

import matplotlib.pyplot as plt
import numpy as np

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a mode

In [20]:
# Load data
# Make sure the file name is correct and in the same directory as the script
#with open("action_type_with_priority_per_session_learning.json") as f:
#    action_type_with_priority_per_session = json.load(f)

#with open("level_3_actions_per_session_learning.json") as f:
#    level_3_actions_per_session_learning = json.load(f)

with open("level_2_actions_per_session_semantic.json") as f:
    level_2_actions_per_session_learning = json.load(f)

pio.renderers.default = "notebook"

Currently sums the frontstage + current backstage if action is backstage and frontstage only if action is fronstage

In [67]:
# Plotting and helper functions
# TODO: Check semantic expansion with reading AI at backstage
# TODO: Parsing still has some deletes not having a level_2_action_type because they often delete text or delete partial prompts
# TODO: Add a point for starting from 0
# TODO: Need to have a backstage to frontstage start writing sort of points
def plot_semantic_expansion(writing_data, session_ids):
    fig = make_subplots(
        rows=len(session_ids),
        cols=1,
        subplot_titles=[f"Session: {session_id}" for session_id in session_ids],
        vertical_spacing=0.15,
    )
    writing_progress_level_2_only = 0
    for idx, session_id in enumerate(session_ids, start=1):
        if session_id not in writing_data:
            print(f"Warning: No data found for {session_id}")
            continue

        session_actions = writing_data[session_id]
        
        learning_data = []
        frontstage_semantic_scores = []
        total_semantic_scores = []
        added_actions = 0
        for i, action in enumerate(session_actions):
          action["writing_progress"] = i + added_actions
          if "level_1_action_type" == None:
            # writing_progress_level_2_only should work too
            learning_data.append(
                    {
                        "writing_progress": writing_progress_level_2_only,
                        "semantic_expansion_score": np.nan,
                        "action_type": action.get("action_type", ""),
                        "cumulative_semantic_expansion_score": sum(frontstage_semantic_scores),
                        "level_1_action_type": "NEXT_CLICKED",
                        "level_2_action_type": "NEXT_CLICKED"
                    }
            )
            writing_progress_level_2_only+=1
          elif "level_2_action_type" in action and "semantic_expansion" in action:
                # TODO: Treating human_edit_ai as frontstage for now
                prev_l2 = i-1
                while prev_l2 > 0:
                    if "writing_type" in session_actions[prev_l2]:
                        break 
                    else:
                        prev_l2-=1
                if prev_l2 > 0 and session_actions[prev_l2]["writing_type"] == "backstage" and action["writing_type"] == "frontstage":
                        learning_data.append(
                        {
                            "writing_progress": writing_progress_level_2_only-1,
                            "semantic_expansion_score": action["semantic_expansion"],
                            "action_type": action.get("action_type", ""),
                            "cumulative_semantic_expansion_score": sum(frontstage_semantic_scores),
                            "level_1_action_type": action.get("level_1_action_type"),
                            "level_2_action_type" : "backstage_frontstage_transition"
                        }
                    )
                        #writing_progress_level_2_only+=1
              
                learning_data.append(
                    {
                        "writing_progress": writing_progress_level_2_only,
                        "semantic_expansion_score": action["semantic_expansion"],
                        "action_type": action.get("action_type", ""),
                        "cumulative_semantic_expansion_score": sum(frontstage_semantic_scores) + action["semantic_expansion"],
                        "level_1_action_type": action.get("level_1_action_type"),
                        "level_2_action_type" : action.get("level_2_action_type")
                    }
                )
                writing_progress_level_2_only+=1
                if action["semantic_expansion"] is not np.nan:
                    total_semantic_scores.append(action["semantic_expansion"])
                    # TODO: Need to check if this works with deleting prompts
                    if "writing_type" in action and action["writing_type"] == "frontstage":
                        frontstage_semantic_scores.append(action["semantic_expansion"])
            
        
        data_df = pd.DataFrame(learning_data)
        field = "cumulative_semantic_expansion_score"
        
        data_df = data_df.dropna(subset=["cumulative_semantic_expansion_score"])
        data_df["wp_valid"]    = data_df["writing_progress"]#.where(data_df["external_score"].notna())
        data_df["score_valid"] = data_df[field]#.where(data_df["external_score"].notna())

        # 2) Shift them up by one row (so that wp_valid.shift(-1)[i] is the value at row i+1)
        data_df["wp_next"]    = data_df["wp_valid"].shift(-1)
        data_df["score_next"] = data_df["score_valid"].shift(-1)

        # 3) Back‐fill: for any row i where step 2 is NaN, fill from the next non‐NaN below
        data_df["wp_next"]    = data_df["wp_next"].bfill()
        data_df["score_next"] = data_df["score_next"].bfill()
        data_df["action_valid"] = data_df["level_2_action_type"]#.where(data_df["external_score"].notna())

        # 2) Shift it up one row so each row points to the following valid action, if it’s immediately next…
        data_df["level_2_action_next"] = data_df["action_valid"].shift(-1)

        # 3) …then back-fill so it jumps over any gaps to the *next* valid action down the table
        data_df["level_2_action_next"] = data_df["level_2_action_next"].bfill()

        data_df["level_2_action_prev"] = data_df["action_valid"].shift(1)
        #print(f"{session_id} — # of validity scores: {len(data_df)}")
        #print(f"{session_id} — # of validity scores: {len(data_df)}")

        colors = {
            "insert_human_backstage": "#53abfc",
            "insert_human_frontstage":"#1302f7",
            "insert_ai_backstage" : "#fa9191",
            "move_frontstage":"#fc0303",
            "human_edit_ai":"#dd77fc",
            "prewrite": "black",
            "backstage_frontstage_transition" : "white"
        }
    

        y_min = min(min(data_df[field]),0)
        y_max = max(data_df[field])

        prewrite_done = [action for action in session_actions if action.get("level_1_action_type") == "NEXT_CLICKED"]
        for action in prewrite_done:
          action_index = action["writing_progress"]
          fig.add_shape(
                  type="line",
                  x0=action_index,
                  x1=action_index,
                  y0=y_min-0.5,
                  y1=y_max+0.5,
                  yref="paper",
                  line=dict(color=colors['prewrite'], width=3,dash="dash"),
                  row=idx,
                  col=1,
              )
        """
        if idx == 1:
            fig.add_trace(
                go.Scatter(
                    x=[None],
                    y=[None],
                    mode="lines",
                    line=dict(color=colors["prewrite"], width=3),
                    name="Prewrite done",
                    showlegend=True,
                ),
                row=1,
                col=1,
            )
        """
    title = ("Semantic Expansion over Writing Sessions<br>"
            + '<span style="font-size: 12px;">*Measuring how much semantic expansion is in people\'s writing while writing with GenAI</span>')

    fig.update_layout(
        plot_bgcolor="white",
        legend_title_font_color="rgba(0, 0, 0, 0)",
        height=350 * len(session_ids),
        showlegend=True,
        title={"text": title},
        width = 1200,
        margin=dict(l=60, r=200, t=100, b=100),
        legend=dict(
            x=1.02,
            y=1,
            xanchor="left"
        )
    )

    fig.update_xaxes(
        title=None,
        mirror=False,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="white",
    )

    fig.update_xaxes(
        title="Progress into the writing session", row=len(session_ids), col=1
    )

    fig.update_yaxes(
        title="Semantic Expansion",
        mirror=False,
        ticks="outside",
        showline=True,
        linecolor="black",
        gridcolor="white",
    )

    
    action_lst = ["insert_human_backstage", "insert_ai_backstage", "move_frontstage", "human_edit_ai","insert_human_frontstage", "backstage_frontstage_transition"]
    for action in action_lst:
        for i, row in data_df[:-1].iterrows():
            if row["level_2_action_next"] == action:
                x0, x1 = row["writing_progress"], row["wp_next"]
                y0, y1 = row[field], row["score_next"]
                fig.add_trace(
                    go.Scatter(
                        x=[x0, x1],
                        y=[y0, y1],
                        mode="lines",
                        line=dict(color=colors[action], width=2),
                        showlegend=False,
                    ),
                    row=idx, col=1
            )
        if action != "backstage_frontstage_transition":
            fig.add_trace(
                go.Scatter(
                    x=[None], y=[None],  # no actual data
                    mode="lines",
                    line=dict(color=colors[action], width=3),
                    name=action   # legend label
                ),
                row=idx,
                col=1,
            )
    fig.show()
    return fig

In [68]:
[
  (action["semantic_expansion"]) for idx,action in enumerate(level_2_actions_per_session_learning["legislation_antitrust_1"]) if "semantic_expansion" in action
]

[0.3788606524467468,
 0.34538193941116335,
 0.356248140335083,
 0.38911616802215576,
 0.39538490772247314,
 0.3690953850746155,
 1.2818763256072998,
 -0.9536346793174744,
 0.4440164566040039,
 0.4885016798973083,
 0.8428242802619934,
 -0.9102123379707336,
 0.7642982006072998,
 -0.010787546634674072,
 0.37167489528656006,
 0.30762001276016226,
 -0.05368238687515259,
 -0.015314221382141113,
 0.9852247834205627,
 0.2473753333091735,
 0.663008451461792,
 0.0,
 -0.39628446102142334,
 0.26480621099472046,
 -0.0361059308052063,
 0.2717057466506958,
 0.25634111166000384,
 0.4446248412132263,
 0.32979739904403704,
 1.481219232082367,
 0.3133211851119997,
 -0.0728265643119812,
 0.0,
 0.00886613130569458,
 0.4589836001396179,
 0.496649527549744,
 0.4898956656455997,
 0.544675350189209,
 0.0,
 0.6084149599075319,
 0.0,
 0.5387180924415587,
 0.35105574131011963,
 0.4404460072517393,
 0.31371279954910314,
 0.6609370708465576,
 0.5260206341743467,
 0.5414647698402408,
 0.5679028272628788,
 0.47672337

In [71]:
# Session IDs: An array of sessions you wish to plot.
# Adjust as needed. Check session IDs in data exploration for options.
session_ids = [
     "legislation_corporate_1",
    "legislation_corporate_2",
    "legislation_corporate_3",
    "legislation_corporate_4",
    "legislation_corporate_5",
    "legislation_corporate_6",
    "legislation_antitrust_1",
    "legislation_antitrust_2",
    "legislation_antitrust_3",
    "legislation_antitrust_4",
    "legislation_antitrust_5",
    "legislation_antitrust_6",
     "legislation_antitrust_7",
      "legislation_antitrust_8",
]
writing_data = level_2_actions_per_session_learning
print(level_2_actions_per_session_learning.keys())
# Generate the plot
for session_id in session_ids:
    fig = plot_semantic_expansion(writing_data, [session_id])

dict_keys(['legislation_corporate_1', 'legislation_corporate_2', 'legislation_corporate_3', 'legislation_corporate_4', 'legislation_corporate_5', 'legislation_corporate_6', 'legislation_antitrust_1', 'legislation_antitrust_2', 'legislation_antitrust_3', 'legislation_antitrust_4', 'legislation_antitrust_5', 'legislation_antitrust_6', 'legislation_antitrust_7', 'legislation_antitrust_8'])
